In [15]:
# 셀 1: 라이브러리 임포트
import json
import re
import pandas as pd

In [16]:
# 셀 2: 파일 경로 설정 (수정 필요)
file_path = r"C:\Users\LEEJUHWAN\Downloads\documents_all_converted.cmds"  # 여기에 실제 파일 경로 입력

In [17]:
# 셀 3: 분석 함수 정의
def count_tags(docBody):
    """docBody에서 style, link 태그 개수 카운트"""
    style_count = len(re.findall(r'<style[^>]*>.*?</style>', docBody, re.DOTALL))
    link_count = len(re.findall(r'<link[^>]*/>', docBody))
    return style_count, link_count



In [19]:
# 셀 4: 파일 읽고 분석
results = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if not line.startswith('addDocument '):
            continue
        
        # JSON 부분 추출
        json_str = line[len('addDocument '):]
        
        # 빈 JSON 스킵
        if json_str == '{}':
            continue
        
        try:
            data = json.loads(json_str)
            doc_num = data.get('docNum', 'N/A')
            doc_body = data.get('docBody', '')
            
            style_count, link_count = count_tags(doc_body)
            
            results.append({
                '줄번호': line_num,
                'docNum': doc_num,
                'style 개수': style_count,
                'link 개수': link_count
            })
        except json.JSONDecodeError as e:
            print(f"줄 {line_num}: JSON 파싱 에러 - {e}")



In [20]:
# 에러 발생한 줄 번호들
error_lines = [8459, 8554, 8581, 8621, 8696, 8703, 8717, 8786, 8808, 8829, 8910, 8915, 8924, 8961, 23318]

# 첫 번째 에러 줄 확인
with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        if i == error_lines[0]:  # 8459번 줄
            json_str = line.strip()[len('addDocument '):]
            
            # 에러 위치 근처 확인 (column 6351 근처)
            error_pos = 6350
            print(f"=== 줄 {i} 에러 위치 근처 (char {error_pos-50} ~ {error_pos+50}) ===")
            print(json_str[error_pos-50:error_pos+50])
            print()
            print("에러 위치:", json_str[error_pos-5:error_pos+5])
            break

=== 줄 8459 에러 위치 근처 (char 6300 ~ 6400) ===
=\"content\" style=\"padding:10px;\"><p><img src=\"/PMS_SITE-U7OI43JLDSMO/approval/attach/approval_2

에러 위치: src=\"/PMS


In [21]:
# 셀 5: 결과 출력
df = pd.DataFrame(results)
print(f"총 문서 수: {len(df)}")
print()
display(df)



총 문서 수: 23320



,줄번호,docNum,style 개수,link 개수
0,1,ANY-사업-791,1,2
1,2,ANY-사업수행-482,1,2
2,3,ANY-경비627,1,2
3,4,ANY-인사-4553,1,2
4,5,ANY-사업수행-485,1,2
...,...,...,...,...
23315,23316,ANY-사업수행-0228,2,0
23316,23317,ANY-경비-847,1,2
23317,23318,ANY-경비-856,1,2
23318,23319,ANY-자금-301,1,2


In [22]:
# 셀 6: 통계 요약
print("\n=== 통계 요약 ===")
print(f"style 태그 - 최소: {df['style 개수'].min()}, 최대: {df['style 개수'].max()}, 평균: {df['style 개수'].mean():.1f}")
print(f"link 태그 - 최소: {df['link 개수'].min()}, 최대: {df['link 개수'].max()}, 평균: {df['link 개수'].mean():.1f}")




=== 통계 요약 ===
style 태그 - 최소: 1, 최대: 3, 평균: 1.3
link 태그 - 최소: 0, 최대: 2, 평균: 2.0


In [24]:
# 셀 7: style 개수별 문서 분포
print("\n=== style 개수별 문서 분포 ===")
print(df['style 개수'].value_counts().sort_index())
print(df['link 개수'].value_counts().sort_index())


=== style 개수별 문서 분포 ===
style 개수
1    15663
2     7643
3       14
Name: count, dtype: int64
link 개수
0       15
2    23305
Name: count, dtype: int64
